# Tool-Calling Refinement with KTO

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ProfSynapse/Toolset-Training/blob/main/Trainers/notebooks/kto_tool_calling_notebook.ipynb)

## 🎓 What You'll Learn

This notebook teaches you how to **refine** a language model using KTO (Kahneman-Tversky Optimization) to prefer better tool-calling behaviors. By the end, you'll have:

- **A refined AI model** that prefers correct tool usage over incorrect patterns
- **Hands-on experience** with preference learning (KTO)
- **Understanding** of how KTO differs from SFT

## 🔬 What is KTO?

**KTO (Kahneman-Tversky Optimization)** is a preference learning method:
- You show the model pairs of good vs bad examples
- The model learns to prefer the good patterns
- Use KTO to **refine** an already-capable model

**When to use KTO:**
- ✅ Refining existing tool-calling behavior
- ✅ Teaching preferences between good/bad outputs
- ✅ Improving behavioral patterns (error handling, verification, etc.)

**Not for:**
- ❌ Teaching tool-calling from scratch (use SFT instead)
- ❌ Learning entirely new task formats (use SFT instead)

## 💻 Hardware Requirements

**Recommended GPU:**
- 1.5B-3B models: T4 (15GB VRAM) - ✅ **Free Colab tier works!**
- 7B models: T4 or better
- 13B+ models: A100 (40GB VRAM) - Colab Pro

**Training time:** ~30-45 minutes for a 1.5B model on T4

## 📊 Dataset

- **Dataset**: claudesidian-synthetic-dataset v1.5
- **File**: behavior_merged_kto_v1.5_11.29.25.jsonl
- **Total Examples**: 4,268 (2,286 desirable, 1,982 undesirable)
- **Ratio**: 1.15:1 (well-balanced for KTO)
- **Behaviors**: 12 categories including error_recovery, intellectual_humility, strategic_tool_selection, verification_before_action, and more
- **Format**: OpenAI-compatible tool calling with KTO interleaving

## 1. Installation

Install Unsloth and dependencies. This takes ~2-3 minutes.

In [ ]:
# Install Unsloth for faster training
%%capture
!pip install unsloth
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

# Install Transformers 5 from special branch (required for Ministral 3)
!pip install git+https://github.com/huggingface/transformers.git@bf3f0ae70d0e902efab4b8517fce88f6697636ce

In [ ]:
# Install training dependencies
%%capture
# TRL 0.22.2 is required for Transformers 5 + Unsloth compatibility
!pip install --no-deps trl==0.22.2
!pip install "datasets>=2.14.0"
!pip install -U accelerate bitsandbytes
!pip install -U peft xformers triton
# Reinstall unsloth_zoo for FastVisionModel support (Ministral 3)
!pip install --upgrade --force-reinstall --no-cache-dir --no-deps unsloth unsloth_zoo

## 2. Mount Google Drive (Optional)

Save checkpoints to Google Drive so they persist if runtime disconnects.

In [ ]:
from google.colab import drive
import os

drive.mount('/content/drive')

# Create output directory
DRIVE_OUTPUT_DIR = "/content/drive/MyDrive/KTO_Training"
os.makedirs(DRIVE_OUTPUT_DIR, exist_ok=True)

print(f"✓ Google Drive mounted")
print(f"✓ Checkpoints will be saved to: {DRIVE_OUTPUT_DIR}")

## 3. HuggingFace Credentials

Add your HF token to Colab secrets:
1. Click the 🔑 key icon in the left sidebar
2. Add new secret: `HF_TOKEN`
3. Get token from https://huggingface.co/settings/tokens

In [ ]:
import os
from google.colab import userdata
from huggingface_hub import HfApi

# Get token from Colab secrets
HF_TOKEN = userdata.get('HF_TOKEN')
os.environ['HF_TOKEN'] = HF_TOKEN

# Get your HuggingFace username automatically
api = HfApi()
hf_user = api.whoami(token=HF_TOKEN)["name"]

print(f"✓ HuggingFace token loaded")
print(f"✓ Username: {hf_user}")

## 4. Model Configuration

**What this does:** Choose the base model you want to refine and configure basic settings.

For KTO, you're typically starting with a model that already has some capability—you're teaching it to prefer better patterns.

In [ ]:
# @title ⚙️ Model & Dataset Configuration
# @markdown Use the dropdowns to select your model and configure your dataset.

# @markdown ### 🧠 Base Model Selection
# @markdown Choose a model based on your VRAM availability. Smaller models recommended for KTO.
# @markdown * **1B-3B:** Fast, runs on any GPU (recommended for free Colab)
# @markdown * **7B-9B:** Standard balance of speed/intelligence
# @markdown * **12B+:** High intelligence, requires A100
# @markdown * **Ministral 3 (NEW):** Latest Mistral model with vision capabilities (3B/8B/14B)
MODEL_NAME = "unsloth/Ministral-3-8B-Instruct-2512" # @param ["unsloth/Llama-3.2-1B-Instruct-bnb-4bit", "unsloth/Qwen2.5-1.5B-Instruct-bnb-4bit", "unsloth/Qwen2.5-Coder-1.5B-Instruct-bnb-4bit", "unsloth/gemma-2-2b-it-bnb-4bit", "unsloth/Ministral-3-3B-Instruct-2512", "unsloth/Llama-3.2-3B-Instruct-bnb-4bit", "unsloth/Qwen2.5-3B-Instruct-bnb-4bit", "unsloth/Phi-3.5-mini-instruct", "unsloth/Qwen2.5-7B-Instruct-bnb-4bit", "unsloth/mistral-7b-instruct-v0.3-bnb-4bit", "unsloth/Ministral-3-8B-Instruct-2512", "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit", "unsloth/Ministral-3-14B-Instruct-2512"]

# @markdown ### 📏 Max Output Length
# @markdown 4096 recommended for tool calling. Higher values require more VRAM.
MAX_SEQ_LENGTH = 4096 # @param [1024, 2048, 4096, 8192] {type:"raw"}

# @markdown ### 📚 Dataset Configuration
DATASET_NAME = "professorsynapse/claudesidian-synthetic-dataset" # @param {type:"string"}
DATASET_FILE = "behavior_merged_kto_v1.5_11.29.25.jsonl" # @param {type:"string"}

# @markdown ### 🏷️ Output Model Name
# @markdown Name your refined model (e.g., `my-tool-model-kto-v1`).
OUTPUT_MODEL_NAME = "nexus-tools-kto" # @param {type:"string"}

# Detect if this is a Ministral 3 model (requires FastVisionModel)
IS_MINISTRAL = "ministral" in MODEL_NAME.lower()

print(f"✓ Configuration set:")
print(f"  • Model: {MODEL_NAME}")
print(f"  • Context: {MAX_SEQ_LENGTH}")
print(f"  • Dataset: {DATASET_NAME}/{DATASET_FILE}")
print(f"  • Output: {OUTPUT_MODEL_NAME}")
if IS_MINISTRAL:
    print(f"  • Note: Ministral 3 detected - will use FastVisionModel")

## 5. Load Model and Tokenizer

**What this does:** Downloads the base model and prepares it for training.

The model is the "brain" that will learn to prefer correct tool-calling patterns. We use 4-bit quantization to fit large models into limited GPU memory.

In [ ]:
from unsloth import FastLanguageModel
import torch

# Check GPU and store info for lineage
GPU_NAME = torch.cuda.get_device_name(0)
CUDA_VERSION = torch.version.cuda
GPU_MEMORY_GB = torch.cuda.get_device_properties(0).total_memory / 1024**3

print(f"Using GPU: {GPU_NAME}")
print(f"CUDA version: {CUDA_VERSION}")
print(f"Available VRAM: {GPU_MEMORY_GB:.1f} GB")
print()

In [ ]:
# Load the base model and tokenizer from HuggingFace
# This downloads the model weights
#
# Parameters explained:
#   model_name: Which model to download
#   max_seq_length: Max tokens model can process at once
#   dtype=None: Auto-detect best precision for your GPU
#   load_in_4bit=True: Use 4-bit quantization to save memory
#   token: Your HF token for accessing the model

# Use FastVisionModel for Ministral 3, FastLanguageModel for others
if IS_MINISTRAL:
    from unsloth import FastVisionModel
    model, tokenizer = FastVisionModel.from_pretrained(
        model_name=MODEL_NAME,
        max_seq_length=MAX_SEQ_LENGTH,
        dtype=None,  # Auto-detect (usually bfloat16 or float16)
        load_in_4bit=True,  # Reduces memory usage by ~75%
        token=HF_TOKEN,
    )
    ModelClass = FastVisionModel
else:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name=MODEL_NAME,
        max_seq_length=MAX_SEQ_LENGTH,
        dtype=None,  # Auto-detect (usually bfloat16 or float16)
        load_in_4bit=True,  # Reduces memory usage by ~75%
        token=HF_TOKEN,
    )
    ModelClass = FastLanguageModel

# ════════════════════════════════════════════════════════════════════════
# CRITICAL: Apply chat template BEFORE training
# ════════════════════════════════════════════════════════════════════════
# This ensures the special tokens (<|im_start|>, <|im_end|>) are properly
# handled as single tokens, not as literal text strings.
# Without this, the model will output "<|im_start|>assistant" as text!

from unsloth.chat_templates import get_chat_template

# Detect the correct chat template based on model name
if "qwen" in MODEL_NAME.lower():
    chat_template_name = "chatml"  # Qwen uses ChatML format
elif "llama" in MODEL_NAME.lower():
    chat_template_name = "llama-3"
elif "mistral" in MODEL_NAME.lower() or "ministral" in MODEL_NAME.lower():
    chat_template_name = "mistral"
elif "gemma" in MODEL_NAME.lower():
    chat_template_name = "gemma"
elif "phi" in MODEL_NAME.lower():
    chat_template_name = "phi-3"
else:
    chat_template_name = "chatml"  # Default fallback

tokenizer = get_chat_template(
    tokenizer,
    chat_template=chat_template_name,
)

print(f"Applied {chat_template_name} chat template")

# Store for lineage
TOTAL_PARAMS = sum(p.numel() for p in model.parameters())
CHAT_TEMPLATE = chat_template_name

print("Model loaded successfully")
print(f"  Model has {TOTAL_PARAMS:,} parameters")
print(f"  Chat template: {chat_template_name}")
if IS_MINISTRAL:
    print(f"  Using: FastVisionModel (Ministral 3)")

## 6. Apply LoRA Adapters

**What this does:** Add trainable "adapter" layers to the model instead of training the entire thing.

Think of LoRA like teaching a new preference through muscle memory—we add small specialized layers that learn to prefer better behaviors, while keeping the main "brain" frozen.

In [ ]:
# @title 🔧 LoRA Adapter Configuration
# @markdown Configure the size and strength of the fine-tuning adapters.

# @markdown ### 🎛️ LoRA Parameters
# @markdown **Rank (r):** Higher = more capacity but slower/more memory (Standard: 32-64 for KTO).
# @markdown Alpha will be automatically set to 2 * r.
LORA_R = 64 # @param [16, 32, 64, 128] {type:"raw"}

LORA_ALPHA = LORA_R * 2

# @markdown **Dropout:** Helps prevent overfitting (Standard: 0.05).
LORA_DROPOUT = 0.05 # @param {type:"number"}

# @markdown **Random Seed:** Change this for different initialization.
RANDOM_STATE = 3407 # @param {type:"integer"}

# Target modules for LoRA
TARGET_MODULES = ["q_proj", "k_proj", "v_proj", "o_proj",
                  "gate_proj", "up_proj", "down_proj"]

# Use the correct model class (FastVisionModel for Ministral, FastLanguageModel for others)
model = ModelClass.get_peft_model(
    model,
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    lora_dropout=LORA_DROPOUT,
    bias="none",
    target_modules=TARGET_MODULES,
    use_gradient_checkpointing="unsloth",
    random_state=RANDOM_STATE,
)

# Store for lineage
TRAINABLE_PARAMS = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"✓ LoRA adapters applied:")
print(f"  • Rank: {LORA_R}")
print(f"  • Alpha: {LORA_ALPHA}")
print(f"  • Dropout: {LORA_DROPOUT}")
print(f"  • Trainable params: {TRAINABLE_PARAMS:,}")

## 7. Load and Prepare Dataset

**What this does:** Downloads training examples and formats them for KTO.

The dataset contains ~1,850 examples of **both good AND bad** tool-calling behavior. KTO learns by comparing:
- ✅ **Desirable examples** (label=true): Correct tool usage
- ❌ **Undesirable examples** (label=false): Common mistakes

This is the key difference from SFT—KTO learns *preferences*, not just *patterns*.

In [ ]:
from datasets import load_dataset, Dataset

"""
═════════════════════════════════════════════════════════════════════════
LOAD DATASET FROM HUGGINGFACE
═════════════════════════════════════════════════════════════════════════
This downloads pre-made training examples with BOTH good and bad tool-calling.
Each example shows: user request → tool call (correct OR incorrect)
"""

print(f"Loading dataset: {DATASET_NAME}/{DATASET_FILE}")
raw_dataset = load_dataset(
    DATASET_NAME,
    data_files=DATASET_FILE,
    split="train"
)

# Store dataset info for lineage
DATASET_SIZE = len(raw_dataset)

# Try to get behaviors list from first few examples
BEHAVIORS = list(set(ex.get('behavior', 'general') for ex in raw_dataset[:100] if 'behavior' in ex))
if not BEHAVIORS:
    BEHAVIORS = ['general']

print(f"✓ Loaded {DATASET_SIZE} examples")
print(f"  Dataset: {DATASET_NAME}/{DATASET_FILE}")
print(f"  Behaviors: {len(BEHAVIORS)} categories")

In [ ]:
"""
═════════════════════════════════════════════════════════════════════════
CONVERT TO KTO FORMAT
═════════════════════════════════════════════════════════════════════════
KTO expects:
  - prompt: The user request
  - completion: The model's response (tool call)
  - label: True (desirable) or False (undesirable)
"""

def convert_to_kto_format(example):
    """
    Convert tool calling conversations to KTO format.
    
    KTO learns by comparing desirable vs undesirable completions
    for the same type of prompt.
    """
    conversations = example["conversations"]
    behavior = example.get("behavior", "general")
    
    # Extract user and assistant messages
    user_msg = None
    assistant_msg = None
    
    for msg in conversations:
        if msg["role"] == "user":
            user_msg = msg["content"]
        elif msg["role"] == "assistant":
            # Handle OpenAI tool calling format
            if "tool_calls" in msg and msg["tool_calls"]:
                tool_calls_str = []
                for tc in msg["tool_calls"]:
                    func = tc["function"]
                    tool_calls_str.append(
                        f"tool_call: {func['name']}\n"
                        f"arguments: {func['arguments']}"
                    )
                assistant_msg = "\n\n".join(tool_calls_str)
            else:
                assistant_msg = msg.get("content", "")
    
    if user_msg is None or assistant_msg is None:
        return None
    
    return {
        "prompt": user_msg,
        "completion": assistant_msg,
        "label": example["label"]
    }

# Process the dataset (raw_dataset is already the train split)
kto_data = []
skipped = 0
for example in raw_dataset:
    processed = convert_to_kto_format(example)
    if processed:
        kto_data.append(processed)
    else:
        skipped += 1

# Create HuggingFace Dataset
train_dataset = Dataset.from_dict({
    "prompt": [ex["prompt"] for ex in kto_data],
    "completion": [ex["completion"] for ex in kto_data],
    "label": [ex["label"] for ex in kto_data],
})

# Calculate and store statistics for lineage
DESIRABLE_COUNT = sum(train_dataset["label"])
UNDESIRABLE_COUNT = len(train_dataset) - DESIRABLE_COUNT
DATASET_PROCESSED_SIZE = len(train_dataset)

print(f"✓ KTO Dataset prepared:")
print(f"  • Total examples: {DATASET_PROCESSED_SIZE}")
print(f"  • Desirable (good): {DESIRABLE_COUNT} ✅")
print(f"  • Undesirable (bad): {UNDESIRABLE_COUNT} ❌")
print(f"  • Ratio: {DESIRABLE_COUNT/UNDESIRABLE_COUNT:.2f}:1")
if skipped > 0:
    print(f"  • Skipped: {skipped}")

print(f"\n📝 Sample desirable example:")
desirable_ex = [ex for ex in kto_data if ex["label"]][0]
print(f"Prompt: {desirable_ex['prompt'][:100]}...")
print(f"Completion: {desirable_ex['completion'][:150]}...")

print(f"\n📝 Sample undesirable example:")
undesirable_ex = [ex for ex in kto_data if not ex["label"]][0]
print(f"Prompt: {undesirable_ex['prompt'][:100]}...")
print(f"Completion: {undesirable_ex['completion'][:150]}...")

## 8. Training Configuration

**What this does:** Set the hyperparameters that control how the model learns preferences.

KTO has a key parameter called **beta** that controls how strongly the model should prefer desirable outputs. Higher beta = stronger preference learning.

**Key differences from SFT:**
- **Lower learning rate** (5e-6 vs 2e-4): Preference learning is more subtle
- **Beta parameter**: Controls KTO's preference strength
- **Desirable/Undesirable weights**: Balance the learning signal

In [ ]:
from trl import KTOConfig, KTOTrainer
from unsloth import is_bfloat16_supported
from datetime import datetime

# Create timestamped output directory
TRAINING_TIMESTAMP = datetime.now().strftime("%Y%m%d_%H%M%S")
output_dir = f"{DRIVE_OUTPUT_DIR}/{TRAINING_TIMESTAMP}"

# @title 🏃 Training Hyperparameters
# @markdown Control the speed and quality of training.

# @markdown ### ⚡ Performance Settings
# @markdown **Batch Size:** Examples per step. Lower if you run out of memory.
BATCH_SIZE = 4 # @param [1, 2, 4, 6, 8] {type:"raw"}

# @markdown **Gradient Accumulation:** Simulates larger batches.
GRADIENT_ACCUMULATION = 8 # @param [2, 4, 6, 8, 12, 16] {type:"raw"}

# @markdown ### 🧠 KTO-Specific Parameters
# @markdown **Beta:** Controls preference strength. Higher = stronger preferences.
# @markdown * **0.05-0.1:** Gentle preference learning
# @markdown * **0.1-0.3:** Standard (recommended)
# @markdown * **0.3-0.5:** Strong preference enforcement
KTO_BETA = 0.1 # @param [0.05, 0.1, 0.2, 0.3, 0.5] {type:"raw"}

# @markdown **Desirable Weight:** Weight for positive examples.
DESIRABLE_WEIGHT = 1.0 # @param {type:"number"}

# @markdown **Undesirable Weight:** Weight for negative examples.
UNDESIRABLE_WEIGHT = 1.0 # @param {type:"number"}

# @markdown ### 📚 Learning Rate Configuration
# @markdown KTO uses lower learning rates than SFT (preference learning is subtle).
# @markdown * **5e-6:** Standard for KTO (recommended)
# @markdown * **1e-6:** Very conservative
# @markdown * **1e-5:** Faster but riskier
LEARNING_RATE_EXPONENT = 6 # @param [5, 6, 7] {type:"raw"}
LEARNING_RATE_MULTIPLIER = 5 # @param [1, 2, 3, 5] {type:"raw"}

LEARNING_RATE = LEARNING_RATE_MULTIPLIER * (10 ** -LEARNING_RATE_EXPONENT)

# @markdown ### 🔄 Epochs
# @markdown Number of passes through the dataset.
NUM_EPOCHS = 3 # @param {type:"integer"}

# @markdown ### 💾 Saving & Logging
SAVE_STEPS = 100 # @param {type:"integer"}
LOGGING_STEPS = 10 # @param {type:"integer"}

# Other training settings
WARMUP_RATIO = 0.1
MAX_GRAD_NORM = 1.0
LR_SCHEDULER = "cosine"
OPTIMIZER = "adamw_8bit"
USE_BF16 = is_bfloat16_supported()

training_args = KTOConfig(
    output_dir=output_dir,
    per_device_train_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION,
    
    # KTO-specific
    beta=KTO_BETA,
    desirable_weight=DESIRABLE_WEIGHT,
    undesirable_weight=UNDESIRABLE_WEIGHT,
    
    # Learning rate
    learning_rate=LEARNING_RATE,
    max_grad_norm=MAX_GRAD_NORM,
    lr_scheduler_type=LR_SCHEDULER,
    warmup_ratio=WARMUP_RATIO,
    
    # Sequence lengths
    max_length=MAX_SEQ_LENGTH,
    max_prompt_length=MAX_SEQ_LENGTH // 2,
    
    # Training schedule
    num_train_epochs=NUM_EPOCHS,
    
    # Memory optimizations
    gradient_checkpointing=True,
    optim=OPTIMIZER,
    fp16=not USE_BF16,
    bf16=USE_BF16,
    
    # Logging and saving
    logging_steps=LOGGING_STEPS,
    save_steps=SAVE_STEPS,
    save_total_limit=3,
    
    # Performance
    dataloader_num_workers=2,
    seed=RANDOM_STATE,
    report_to="none",
)

# Calculate effective batch size
EFFECTIVE_BATCH_SIZE = BATCH_SIZE * GRADIENT_ACCUMULATION

print("✓ Training configuration ready")
print(f"  • Batch Size: {BATCH_SIZE}")
print(f"  • Gradient Accum.: {GRADIENT_ACCUMULATION}")
print(f"  • Effective Batch: {EFFECTIVE_BATCH_SIZE}")
print(f"  • Learning Rate: {LEARNING_RATE} ({LEARNING_RATE_MULTIPLIER}e-{LEARNING_RATE_EXPONENT})")
print(f"  • KTO Beta: {KTO_BETA}")
print(f"  • Epochs: {NUM_EPOCHS}")
print(f"  • Output Dir: {output_dir}")

## 9. Initialize Trainer

**What this does:** Creates the KTO training engine that coordinates everything.

The KTOTrainer is similar to SFTTrainer but optimizes for *preferences* rather than just *imitation*.

In [ ]:
# Create the KTOTrainer
# This combines the model, dataset, and configuration into one training pipeline

kto_trainer = KTOTrainer(
    model=model,  # The model with LoRA adapters
    args=training_args,  # All the hyperparameters we configured
    processing_class=tokenizer,  # For converting text to tokens
    train_dataset=train_dataset,  # Our formatted training examples
)

print("✓ KTO Trainer initialized")
print("  Ready to start training!")

## 10. Train!

**What this does:** The actual preference learning happens here!

The model will:
1. **Read examples** of good AND bad tool usage
2. **Compare** desirable vs undesirable completions
3. **Update weights** to prefer the desirable patterns
4. **Repeat** this process for 3 epochs

**What to expect:**
- Training takes ~30-45 minutes for 1.5B models on T4 GPU
- You'll see progress updates every 10 steps
- Loss should generally decrease over time
- Checkpoints are saved every 100 steps to Google Drive

**What the metrics mean:**
- **Loss:** How well the model separates good from bad (lower = better)
- **Learning Rate:** Gradually decreases as training progresses
- **Rewards/chosen:** How much the model prefers good examples (higher = better)
- **Rewards/rejected:** How much the model avoids bad examples (lower = better)

**💾 Checkpoint Resumption:**
If your Colab session disconnects, your checkpoints are saved to Google Drive. Re-run the setup cells and training will automatically resume.

In [ ]:
import torch
import glob
import os
import time

# Check GPU memory
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")
print()

# ════════════════════════════════════════════════════════════════════════
# 🔍 Check for existing checkpoints (automatic resumption)
# ════════════════════════════════════════════════════════════════════════
checkpoint_dirs = sorted(glob.glob(f"{output_dir}/checkpoint-*"))
resume_from_checkpoint = None

if checkpoint_dirs:
    # Found checkpoints - get the latest one
    latest_checkpoint = max(checkpoint_dirs, key=lambda x: int(x.split("-")[-1]))
    resume_from_checkpoint = latest_checkpoint
    print(f"✓ Found existing checkpoint: {os.path.basename(latest_checkpoint)}")
    print(f"  Resuming training from this checkpoint")
    print(f"  Total checkpoints found: {len(checkpoint_dirs)}")
else:
    print("ℹ️  No existing checkpoints found - starting fresh training")

print()

# Start training (or resume)
print("=" * 60)
if resume_from_checkpoint:
    print("RESUMING KTO TRAINING FROM CHECKPOINT")
else:
    print("STARTING KTO TRAINING")
print("=" * 60)
print()

# Track training time
training_start_time = time.time()

trainer_stats = kto_trainer.train(resume_from_checkpoint=resume_from_checkpoint)

# Calculate training duration
TRAINING_DURATION_SECONDS = time.time() - training_start_time
TRAINING_DURATION_MINUTES = TRAINING_DURATION_SECONDS / 60

# Store final metrics for lineage
FINAL_LOSS = trainer_stats.training_loss
TOTAL_STEPS = trainer_stats.global_step

print()
print("=" * 60)
print("TRAINING COMPLETED")
print("=" * 60)
print(f"Final loss: {FINAL_LOSS:.4f}")
print(f"Total steps: {TOTAL_STEPS}")
print(f"Training time: {TRAINING_DURATION_MINUTES:.1f} minutes")

## 11. Build Training Lineage

**What this does:** Captures all training metadata for reproducibility and analysis.

This creates a complete record of:
- Base model and configuration
- Dataset details and statistics
- All hyperparameters used
- Training results and metrics
- Hardware and environment info

This information will be automatically added to your HuggingFace model card!

In [ ]:
import json
from datetime import datetime

"""
BUILD COMPLETE TRAINING LINEAGE

This dictionary captures EVERYTHING about the training run for:
- Reproducibility
- Model card generation
- Experiment tracking
- Analysis and comparison
"""

TRAINING_LINEAGE = {
    # IDENTIFICATION
    "model_name": OUTPUT_MODEL_NAME,
    "training_method": "KTO",
    "training_timestamp": TRAINING_TIMESTAMP,
    "training_date": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    
    # BASE MODEL
    "base_model": {
        "name": MODEL_NAME,
        "total_parameters": TOTAL_PARAMS,
        "quantization": "4-bit",
        "max_seq_length": MAX_SEQ_LENGTH,
        "chat_template": CHAT_TEMPLATE,  # Added chat template tracking
    },
    
    # LORA CONFIGURATION
    "lora_config": {
        "r": LORA_R,
        "alpha": LORA_ALPHA,
        "dropout": LORA_DROPOUT,
        "target_modules": TARGET_MODULES,
        "trainable_parameters": TRAINABLE_PARAMS,
        "trainable_percentage": round(TRAINABLE_PARAMS / TOTAL_PARAMS * 100, 4),
    },
    
    # DATASET
    "dataset": {
        "name": DATASET_NAME,
        "huggingface_url": f"https://huggingface.co/datasets/{DATASET_NAME}",
        "total_examples": DATASET_PROCESSED_SIZE,
        "desirable_examples": DESIRABLE_COUNT,
        "undesirable_examples": UNDESIRABLE_COUNT,
        "desirable_ratio": round(DESIRABLE_COUNT / UNDESIRABLE_COUNT, 2),
        "behaviors": sorted(BEHAVIORS),
    },
    
    # KTO HYPERPARAMETERS
    "kto_config": {
        "beta": KTO_BETA,
        "desirable_weight": DESIRABLE_WEIGHT,
        "undesirable_weight": UNDESIRABLE_WEIGHT,
    },
    
    # TRAINING HYPERPARAMETERS
    "training_config": {
        "batch_size": BATCH_SIZE,
        "gradient_accumulation_steps": GRADIENT_ACCUMULATION,
        "effective_batch_size": EFFECTIVE_BATCH_SIZE,
        "learning_rate": LEARNING_RATE,
        "learning_rate_scheduler": LR_SCHEDULER,
        "warmup_ratio": WARMUP_RATIO,
        "max_grad_norm": MAX_GRAD_NORM,
        "num_epochs": NUM_EPOCHS,
        "optimizer": OPTIMIZER,
        "precision": "bf16" if USE_BF16 else "fp16",
        "gradient_checkpointing": True,
        "random_seed": RANDOM_STATE,
    },
    
    # TRAINING RESULTS
    "training_results": {
        "final_loss": round(FINAL_LOSS, 4),
        "total_steps": TOTAL_STEPS,
        "training_duration_minutes": round(TRAINING_DURATION_MINUTES, 1),
    },
    
    # HARDWARE & ENVIRONMENT
    "hardware": {
        "gpu": GPU_NAME,
        "gpu_memory_gb": round(GPU_MEMORY_GB, 1),
        "cuda_version": CUDA_VERSION,
        "platform": "Google Colab",
    },
    
    # FRAMEWORK VERSIONS
    "framework_versions": {
        "torch": torch.__version__,
        "transformers": __import__("transformers").__version__,
        "trl": __import__("trl").__version__,
        "peft": __import__("peft").__version__,
        "unsloth": "latest",
    },
}

# Save lineage to file
lineage_path = f"{output_dir}/training_lineage.json"
with open(lineage_path, "w") as f:
    json.dump(TRAINING_LINEAGE, f, indent=2)

print("Training lineage captured!")
print(f"  Saved to: {lineage_path}")
print()
print("Summary:")
print(f"  Base Model: {MODEL_NAME}")
print(f"  Chat Template: {CHAT_TEMPLATE}")
print(f"  Dataset: {DATASET_NAME} ({DATASET_PROCESSED_SIZE} examples)")
print(f"  Method: KTO (beta={KTO_BETA})")
print(f"  LoRA: r={LORA_R}, alpha={LORA_ALPHA}")
print(f"  LR: {LEARNING_RATE}, Epochs: {NUM_EPOCHS}")
print(f"  Final Loss: {FINAL_LOSS:.4f}")
print(f"  Duration: {TRAINING_DURATION_MINUTES:.1f} min on {GPU_NAME}")

## 12. Upload to HuggingFace

**What this does:** Share your refined model with the world!

The model card will be **automatically generated** with all your training details:
- Base model and configuration
- Dataset information
- All hyperparameters
- Training results
- Hardware used

We'll create **three versions** of your model:

1. **LoRA adapters** - Small files that contain just the preference changes
2. **Merged 16-bit model** - Full model with adapters merged in
3. **GGUF quantizations** - Optimized versions for CPU/GPU inference

In [ ]:
def generate_model_card(lineage: dict, hf_username: str) -> str:
    """
    Generate a comprehensive HuggingFace model card from training lineage.
    
    This creates a professional README.md with all training details
    for reproducibility and transparency.
    """
    
    base_model = lineage["base_model"]["name"]
    dataset = lineage["dataset"]
    lora = lineage["lora_config"]
    kto = lineage["kto_config"]
    training = lineage["training_config"]
    results = lineage["training_results"]
    hardware = lineage["hardware"]
    frameworks = lineage["framework_versions"]
    
    # Format behaviors list
    behaviors_list = "\n".join([f"  - {b}" for b in dataset["behaviors"]])
    
    model_card = f'''---
language:
- en
license: apache-2.0
library_name: transformers
tags:
- tool-calling
- kto
- preference-learning
- claudesidian
- obsidian
- fine-tuned
- unsloth
base_model: {base_model}
datasets:
- {dataset["name"]}
pipeline_tag: text-generation
model-index:
- name: {lineage["model_name"]}
  results:
  - task:
      type: text-generation
    metrics:
    - name: Final Loss
      type: loss
      value: {results["final_loss"]}
---

# {lineage["model_name"]}

This model was fine-tuned using **KTO (Kahneman-Tversky Optimization)** to improve tool-calling behavior for the Claudesidian vault application.

## Model Description

- **Base Model:** [{base_model}](https://huggingface.co/{base_model})
- **Training Method:** KTO (Preference Learning)
- **Task:** Tool-calling for Obsidian vault operations
- **Training Date:** {lineage["training_date"]}

## Training Details

### Dataset

| Property | Value |
|----------|-------|
| Dataset | [{dataset["name"]}]({dataset["huggingface_url"]}) |
| Total Examples | {dataset["total_examples"]:,} |
| Desirable (✅) | {dataset["desirable_examples"]:,} |
| Undesirable (❌) | {dataset["undesirable_examples"]:,} |
| Ratio | {dataset["desirable_ratio"]}:1 |

**Behaviors trained:**
{behaviors_list}

### KTO Configuration

| Parameter | Value |
|-----------|-------|
| Beta (β) | {kto["beta"]} |
| Desirable Weight | {kto["desirable_weight"]} |
| Undesirable Weight | {kto["undesirable_weight"]} |

### LoRA Configuration

| Parameter | Value |
|-----------|-------|
| Rank (r) | {lora["r"]} |
| Alpha (α) | {lora["alpha"]} |
| Dropout | {lora["dropout"]} |
| Target Modules | {', '.join(lora["target_modules"])} |
| Trainable Parameters | {lora["trainable_parameters"]:,} ({lora["trainable_percentage"]}%) |

### Training Hyperparameters

| Parameter | Value |
|-----------|-------|
| Batch Size | {training["batch_size"]} |
| Gradient Accumulation | {training["gradient_accumulation_steps"]} |
| Effective Batch Size | {training["effective_batch_size"]} |
| Learning Rate | {training["learning_rate"]} |
| LR Scheduler | {training["learning_rate_scheduler"]} |
| Warmup Ratio | {training["warmup_ratio"]} |
| Max Grad Norm | {training["max_grad_norm"]} |
| Epochs | {training["num_epochs"]} |
| Optimizer | {training["optimizer"]} |
| Precision | {training["precision"]} |
| Random Seed | {training["random_seed"]} |

### Training Results

| Metric | Value |
|--------|-------|
| Final Loss | {results["final_loss"]} |
| Total Steps | {results["total_steps"]:,} |
| Training Duration | {results["training_duration_minutes"]} minutes |

### Hardware

| Component | Value |
|-----------|-------|
| GPU | {hardware["gpu"]} |
| GPU Memory | {hardware["gpu_memory_gb"]} GB |
| CUDA Version | {hardware["cuda_version"]} |
| Platform | {hardware["platform"]} |

### Framework Versions

| Library | Version |
|---------|--------|
| PyTorch | {frameworks["torch"]} |
| Transformers | {frameworks["transformers"]} |
| TRL | {frameworks["trl"]} |
| PEFT | {frameworks["peft"]} |
| Unsloth | {frameworks["unsloth"]} |

## Usage

### With Transformers

```python
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("{hf_username}/{lineage["model_name"]}")
tokenizer = AutoTokenizer.from_pretrained("{hf_username}/{lineage["model_name"]}")

# Example tool-calling prompt
messages = [{{
    "role": "user",
    "content": "Show me the contents of my project roadmap file."
}}]

inputs = tokenizer.apply_chat_template(messages, return_tensors="pt")
outputs = model.generate(inputs, max_new_tokens=512)
print(tokenizer.decode(outputs[0]))
```

### With Ollama (GGUF)

```bash
# Download the GGUF version
ollama pull {hf_username}/{lineage["model_name"]}

# Run inference
ollama run {hf_username}/{lineage["model_name"]}
```

### With LM Studio

1. Open LM Studio → "Discover" tab
2. Search for `{hf_username}/{lineage["model_name"]}`
3. Download the Q4_K_M or Q5_K_M GGUF version
4. Load and test with tool-calling prompts

## Intended Use

This model is designed for:
- Tool-calling in Obsidian vault management applications
- Claudesidian MCP integration
- Local AI assistants that interact with note-taking systems

## Limitations

- Trained specifically for Claudesidian tool schemas
- May not generalize to other tool-calling formats
- Best performance with the specific tool set it was trained on

## Citation

```bibtex
@misc{{{lineage["model_name"].replace("-", "_")},
  author = {{{hf_username}}},
  title = {{{lineage["model_name"]}: KTO Fine-tuned Tool-Calling Model}},
  year = {{2025}},
  publisher = {{HuggingFace}},
  url = {{https://huggingface.co/{hf_username}/{lineage["model_name"]}}}
}}
```

## Training Lineage

<details>
<summary>Click to expand full training configuration (JSON)</summary>

```json
{json.dumps(lineage, indent=2)}
```

</details>
'''
    
    return model_card

# Generate model card
MODEL_CARD = generate_model_card(TRAINING_LINEAGE, hf_user)

# Save model card locally
model_card_path = f"{output_dir}/README.md"
with open(model_card_path, "w") as f:
    f.write(MODEL_CARD)

print("✓ Model card generated!")
print(f"  Saved to: {model_card_path}")
print()
print("Preview (first 50 lines):")
print("=" * 60)
print("\n".join(MODEL_CARD.split("\n")[:50]))
print("...")

In [ ]:
from huggingface_hub import HfApi, upload_file

# Upload LoRA adapters with model card
print("Uploading LoRA adapters...")

model.push_to_hub(
    f"{hf_user}/{OUTPUT_MODEL_NAME}",
    token=HF_TOKEN,
    private=False
)
tokenizer.push_to_hub(
    f"{hf_user}/{OUTPUT_MODEL_NAME}",
    token=HF_TOKEN,
    private=False
)

# Upload model card (README.md)
api = HfApi()
api.upload_file(
    path_or_fileobj=model_card_path,
    path_in_repo="README.md",
    repo_id=f"{hf_user}/{OUTPUT_MODEL_NAME}",
    token=HF_TOKEN,
)

# Upload training lineage JSON
api.upload_file(
    path_or_fileobj=lineage_path,
    path_in_repo="training_lineage.json",
    repo_id=f"{hf_user}/{OUTPUT_MODEL_NAME}",
    token=HF_TOKEN,
)

print(f"✓ LoRA adapters uploaded to HuggingFace")
print(f"✓ Model card with full lineage uploaded")
print(f"  View at: https://huggingface.co/{hf_user}/{OUTPUT_MODEL_NAME}")

In [ ]:
# Upload merged 16-bit model with model card
print("Merging LoRA weights into base model (16-bit)...")
print("This will take ~5 minutes...")

model.push_to_hub_merged(
    f"{hf_user}/{OUTPUT_MODEL_NAME}-merged",
    tokenizer,
    save_method="merged_16bit",
    token=HF_TOKEN,
    private=False
)

# Upload model card to merged repo too
api.upload_file(
    path_or_fileobj=model_card_path,
    path_in_repo="README.md",
    repo_id=f"{hf_user}/{OUTPUT_MODEL_NAME}-merged",
    token=HF_TOKEN,
)

api.upload_file(
    path_or_fileobj=lineage_path,
    path_in_repo="training_lineage.json",
    repo_id=f"{hf_user}/{OUTPUT_MODEL_NAME}-merged",
    token=HF_TOKEN,
)

print(f"✓ Merged model uploaded to HuggingFace")
print(f"✓ Model card with full lineage uploaded")
print(f"  View at: https://huggingface.co/{hf_user}/{OUTPUT_MODEL_NAME}-merged")

In [ ]:
# Create GGUF quantizations
quantization_methods = ["q4_k_m", "q5_k_m", "q8_0"]

print("Creating GGUF quantizations...")
print(f"This will create {len(quantization_methods)} versions")
print()

model.push_to_hub_gguf(
    f"{hf_user}/{OUTPUT_MODEL_NAME}",
    tokenizer,
    quantization_method=quantization_methods,
    token=HF_TOKEN,
)

print()
print("✓ GGUF quantizations created and uploaded!")
print(f"  View at: https://huggingface.co/{hf_user}/{OUTPUT_MODEL_NAME}")

## 13. Evaluate Model (Optional)

**What this does:** Run automated tests to measure your model's tool-calling accuracy.

This will:
- Load your trained model with vLLM for fast inference
- Run test prompts covering all 47 tools
- Calculate pass rates by category
- Generate evaluation lineage that can be added to your model card

**Skip this section** if you want to evaluate later using the standalone evaluation notebook.

In [ ]:
# @title Run Evaluation (Optional)
# @markdown Test your refined model's tool-calling accuracy and behavioral patterns.

# @markdown ### Enable Evaluation
run_evaluation = True # @param {type:"boolean"}

# @markdown ### Test Suite Selection
# @markdown * **Tool Coverage (47 tools):** Tests each tool individually
# @markdown * **Behavioral Patterns (24 tests):** Tests context efficiency, executePrompt delegation, etc.
# @markdown * **All Suites:** Runs both tool coverage and behavioral patterns
eval_test_suite = "All Suites" # @param ["Tool Coverage (47 tools)", "Behavioral Patterns (24 tests)", "All Suites"]

if run_evaluation:
    print("Installing vLLM for evaluation...")
    import subprocess
    subprocess.run(["pip", "install", "-q", "vllm>=0.6.0"], check=True)
    
    # Download evaluator framework
    import requests
    from pathlib import Path
    
    os.makedirs("Evaluator/prompts", exist_ok=True)
    os.makedirs("Evaluator/results", exist_ok=True)
    os.makedirs("tools", exist_ok=True)
    
    REPO_BASE = "https://raw.githubusercontent.com/ProfSynapse/Toolset-Training/main"
    
    eval_files = {
        "Evaluator/__init__.py": "Evaluator/__init__.py",
        "Evaluator/runner.py": "Evaluator/runner.py",
        "Evaluator/schema_validator.py": "Evaluator/schema_validator.py",
        "Evaluator/prompt_sets.py": "Evaluator/prompt_sets.py",
        "Evaluator/reporting.py": "Evaluator/reporting.py",
        "Evaluator/config.py": "Evaluator/config.py",
        "Evaluator/prompts/tool_prompts.json": "Evaluator/prompts/tool_prompts.json",
        "Evaluator/prompts/baseline.json": "Evaluator/prompts/baseline.json",
        "Evaluator/prompts/behavioral_patterns.json": "Evaluator/prompts/behavioral_patterns.json",
        "tools/tool_schemas.json": "tools/tool_schemas.json",
    }
    
    print("Downloading evaluation framework...")
    for remote_path, local_path in eval_files.items():
        url = f"{REPO_BASE}/{remote_path}"
        try:
            response = requests.get(url, timeout=30)
            response.raise_for_status()
            Path(local_path).parent.mkdir(parents=True, exist_ok=True)
            with open(local_path, 'w', encoding='utf-8') as f:
                f.write(response.text)
        except Exception as e:
            print(f"  Warning: Failed to download {remote_path}")
    
    print("Evaluation framework ready")
else:
    print("Evaluation skipped. Set run_evaluation = True to enable.")

In [ ]:
if run_evaluation:
    from vllm import LLM, SamplingParams
    from dataclasses import dataclass
    from typing import Any, Dict, Mapping, Sequence
    import time
    import sys
    
    sys.path.insert(0, '/content')
    
    # Use the merged model for evaluation
    EVAL_MODEL = f"{hf_user}/{OUTPUT_MODEL_NAME}-merged"
    
    print(f"Loading model for evaluation: {EVAL_MODEL}")
    print("This may take 1-2 minutes...")
    
    # Initialize vLLM
    eval_llm = LLM(
        model=EVAL_MODEL,
        tensor_parallel_size=1,
        gpu_memory_utilization=0.85,
        max_model_len=2048,
        trust_remote_code=True,
        dtype="auto",
        token=HF_TOKEN,
    )
    
    # Get the vLLM tokenizer for proper chat template handling
    eval_tokenizer = eval_llm.get_tokenizer()
    
    # Create vLLM client for evaluator
    @dataclass
    class VLLMResponse:
        message: str
        raw: Dict[str, Any]
        latency_s: float
    
    class VLLMClient:
        def __init__(self, llm, llm_tokenizer, temperature=0.2, max_tokens=1024, seed=42):
            self.llm = llm
            self.tokenizer = llm_tokenizer
            self.temperature = temperature
            self.max_tokens = max_tokens
            self.seed = seed
        
        def chat(self, messages):
            # Use the tokenizer's chat template instead of manual construction
            # This ensures special tokens are handled correctly
            prompt = self.tokenizer.apply_chat_template(
                messages,
                tokenize=False,
                add_generation_prompt=True
            )
            
            sampling_params = SamplingParams(
                temperature=self.temperature,
                max_tokens=self.max_tokens,
                seed=self.seed,
            )
            
            start = time.perf_counter()
            outputs = self.llm.generate([prompt], sampling_params)
            latency_s = time.perf_counter() - start
            
            output = outputs[0]
            message = output.outputs[0].text.strip()
            
            return VLLMResponse(
                message=message,
                raw={"output": message},
                latency_s=latency_s
            )
    
    eval_client = VLLMClient(eval_llm, eval_tokenizer)
    print("Model loaded for evaluation")
    
    # Run evaluation
    from Evaluator.prompt_sets import load_prompt_cases, filter_prompts
    from Evaluator.runner import evaluate_cases
    from Evaluator.reporting import build_evaluation_lineage, generate_evaluation_model_card_section
    from Evaluator.config import PromptFilter
    
    # Map test suite to files - updated with behavioral patterns
    eval_suite_map = {
        "Tool Coverage (47 tools)": ["Evaluator/prompts/tool_prompts.json"],
        "Behavioral Patterns (24 tests)": ["Evaluator/prompts/behavioral_patterns.json"],
        "All Suites": [
            "Evaluator/prompts/tool_prompts.json",
            "Evaluator/prompts/behavioral_patterns.json"
        ],
    }
    
    eval_prompt_files = eval_suite_map[eval_test_suite]
    all_eval_records = []
    suite_results = {}  # Track results per suite
    
    print()
    print("=" * 60)
    print("RUNNING EVALUATION")
    print(f"Test Suite: {eval_test_suite}")
    print("=" * 60)
    
    for prompt_file in eval_prompt_files:
        suite_name = prompt_file.split("/")[-1].replace(".json", "")
        cases = load_prompt_cases(prompt_file)
        print(f"\nRunning {len(cases)} tests from {suite_name}")
        
        records = evaluate_cases(
            cases=cases,
            client=eval_client,
            dry_run=False,
        )
        all_eval_records.extend(records)
        
        passed = sum(1 for r in records if r.passed)
        suite_results[suite_name] = {
            "passed": passed,
            "total": len(records),
            "rate": passed/len(records)*100
        }
        print(f"   Results: {passed}/{len(records)} passed ({passed/len(records)*100:.1f}%)")
    
    # Calculate overall results
    eval_passed = sum(1 for r in all_eval_records if r.passed)
    eval_total = len(all_eval_records)
    EVAL_PASS_RATE = round(eval_passed / eval_total * 100, 1)
    
    print()
    print("=" * 60)
    print("EVALUATION COMPLETE")
    print("=" * 60)
    print(f"\nResults by Suite:")
    for suite_name, results in suite_results.items():
        print(f"  {suite_name}: {results['passed']}/{results['total']} ({results['rate']:.1f}%)")
    print(f"\nOverall: {eval_passed}/{eval_total} passed ({EVAL_PASS_RATE}%)")
    
    # Build evaluation lineage
    eval_config = {"temperature": 0.2, "max_tokens": 1024, "seed": 42}
    eval_hardware = {
        "gpu": GPU_NAME,
        "gpu_memory_gb": round(GPU_MEMORY_GB, 1),
        "platform": "Google Colab",
    }
    
    EVALUATION_LINEAGE = build_evaluation_lineage(
        records=all_eval_records,
        model_name=EVAL_MODEL,
        test_suites=eval_prompt_files,
        eval_config=eval_config,
        hardware_info=eval_hardware,
    )
    
    # Add suite-level breakdown to lineage
    EVALUATION_LINEAGE["suite_results"] = suite_results
    
    MODEL_CARD_EVAL_SECTION = generate_evaluation_model_card_section(EVALUATION_LINEAGE)
    
    # Save evaluation lineage
    eval_lineage_path = f"{output_dir}/evaluation_lineage.json"
    with open(eval_lineage_path, "w") as f:
        json.dump(EVALUATION_LINEAGE, f, indent=2)
    
    print()
    print(f"Evaluation lineage saved: {eval_lineage_path}")
    print(f"  Overall Pass Rate: {EVAL_PASS_RATE}%")

In [ ]:
if run_evaluation:
    import re
    import tempfile
    from huggingface_hub import hf_hub_download
    
    print("Uploading evaluation results to HuggingFace...")
    
    # Upload to both LoRA and merged repos
    repos_to_update = [
        f"{hf_user}/{OUTPUT_MODEL_NAME}",
        f"{hf_user}/{OUTPUT_MODEL_NAME}-merged",
    ]
    
    for repo_id in repos_to_update:
        try:
            # Upload evaluation lineage JSON
            api.upload_file(
                path_or_fileobj=eval_lineage_path,
                path_in_repo="evaluation_lineage.json",
                repo_id=repo_id,
                token=HF_TOKEN,
            )
            
            # Download and update README with evaluation section
            try:
                readme_path = hf_hub_download(repo_id=repo_id, filename="README.md", token=HF_TOKEN)
                with open(readme_path, 'r') as f:
                    existing_readme = f.read()
                
                if "## Evaluation Results" in existing_readme:
                    pattern = r'## Evaluation Results.*?(?=\n## |\Z)'
                    updated_readme = re.sub(pattern, MODEL_CARD_EVAL_SECTION, existing_readme, flags=re.DOTALL)
                else:
                    updated_readme = existing_readme.rstrip() + "\n\n" + MODEL_CARD_EVAL_SECTION
                
                with tempfile.NamedTemporaryFile(mode='w', suffix='.md', delete=False) as f:
                    f.write(updated_readme)
                    temp_readme = f.name
                
                api.upload_file(
                    path_or_fileobj=temp_readme,
                    path_in_repo="README.md",
                    repo_id=repo_id,
                    token=HF_TOKEN,
                )
                print(f"  ✓ Updated: {repo_id}")
            except Exception as e:
                print(f"  ⚠️  Could not update README for {repo_id}: {e}")
        
        except Exception as e:
            print(f"  ⚠️  Failed to update {repo_id}: {e}")
    
    print()
    print("=" * 60)
    print("✓ EVALUATION RESULTS UPLOADED")
    print("=" * 60)
    print(f"Pass Rate: {EVAL_PASS_RATE}%")
    print(f"View model card: https://huggingface.co/{hf_user}/{OUTPUT_MODEL_NAME}-merged")

## 14. Test Your Model (Optional)

**What this does:** Quick inference test to see how your refined model behaves.

In [ ]:
from transformers import TextStreamer

# Set up for inference
ModelClass.for_inference(model)

# Note: Chat template was already applied in cell-12 (before training)
# No need to apply it again here

def test_tool_calling(user_message):
    """Generate tool call for a user request."""
    print("\n" + "="*60)
    print("USER REQUEST:")
    print("="*60)
    print(user_message)
    print("\n" + "-"*60)
    print("MODEL RESPONSE:")
    print("-"*60)
    
    messages = [{"role": "user", "content": user_message}]
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to("cuda")
    
    text_streamer = TextStreamer(tokenizer, skip_special_tokens=True, skip_prompt=True)
    outputs = model.generate(
        input_ids=inputs,
        streamer=text_streamer,
        temperature=0.1,
        max_new_tokens=512,
        use_cache=True
    )
    print("\n")

print("Inference setup complete")

In [ ]:
# Test cases covering different Claudesidian behaviors
test_cases = [
    # Content reading
    "Show me the contents of my project roadmap file.",
    
    # Content modification  
    "Add a header to my meeting notes saying 'Q1 2025 Planning'.",
    
    # File operations
    "Delete the old draft file called 'temp-notes.md'.",
    
    # Search operations
    "Find all notes that mention 'product launch'.",
]

print("Testing tool calling with refined model...\n")
for test_case in test_cases:
    test_tool_calling(test_case)

## Done!

Your model has been refined with KTO, evaluated, and uploaded to HuggingFace!

### What You Accomplished

| Step | Description |
|------|-------------|
| ✅ Refined | Trained with KTO to prefer correct tool-calling patterns |
| ✅ Lineage | Captured complete training metadata for reproducibility |
| ✅ Model Card | Auto-generated with all hyperparameters |
| ✅ Formats | Created LoRA, merged 16-bit, and GGUF versions |
| ✅ Evaluation | Tested tool-calling accuracy (if enabled) |
| ✅ Published | Model card includes training AND evaluation results |

### Your Lineage Files

| File | Location | Description |
|------|----------|-------------|
| `training_lineage.json` | HuggingFace + Google Drive | All KTO training parameters |
| `evaluation_lineage.json` | HuggingFace + Google Drive | Test results by category |
| `README.md` | HuggingFace | Auto-generated model card |

### KTO vs SFT: When to Use Each

| Method | Use Case | Learning Type |
|--------|----------|---------------|
| **SFT** | Teaching NEW skills (tool-calling syntax) | Imitation learning |
| **KTO** | Refining EXISTING behavior (prefer better patterns) | Preference learning |

**Typical pipeline:**
1. SFT first → Teach the model *what* tool calling is
2. KTO second → Refine *which* patterns are preferred

### Next Steps

**Test locally with LM Studio:**
1. Open LM Studio → "Discover" tab
2. Search for your model name
3. Download the GGUF version
4. Test with tool-calling prompts

**Test with Ollama:**
```bash
ollama pull {your-username}/{model-name}
ollama run {your-username}/{model-name}
```

---

**Questions?** Check the [Evaluator README](https://github.com/ProfSynapse/Toolset-Training/blob/main/Evaluator/README.md) or open an issue on GitHub.